This script calculates the non-green, likely mined area in counties in our [Region of Interest](https://docs.google.com/document/d/10Kd-6qcMn3c0S7YfY2aAZJZ8eJi1Q0oRvd5tea-ykUM/edit) for 1984 to 2015, and saves the output to county_areas.csv. It takes about two days to run.

I also printed all the results to the console so that I wouldn't lose any of them if the script failed, which is why this notebook is so long. 



In [1]:
import ee
import datetime
import time
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.gridspec as gridspec
%matplotlib inline

/Users/David/Desktop/Jobs/GlobalFishingWatch/github/vessel-maps/utilities/pipa_paper/venv/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
ee.Initialize()

In [4]:
wv_counties = {
    'Boone':54005,
    'Logan':54045,
   'Kanawha':54039,
    'McDowell':54047,
    'Lincoln': 54043
}


In [13]:
NDVI_Threshold = 0.51;

mask_input_60m_2015 = ee.Image('users/jerrilyn/2015mask-PM-fullstudy-area');
# Get the link here: https:#drive.google.com/file/d/0B_MArPTqurHudFp6STU4ZzJHRmc/view

counties = ee.FeatureCollection('ft:1S4EB6319wWW2sWQDPhDvmSBIVrD3iEmCLYB7nMM');


# NDVI_Threshold = {
#   1974: 0.4699,   1977: 0.5950,   1980: 0.5749,   1982: 0.5375,   1984: 0.5154,
#   1985: 0.5120,   1986: 0.5425,   1987: 0.5199,   1988: 0.5290,   1989: 0.4952,
#   1990: 0.5022,   1991: 0.4940,   1992: 0.4892,   1993: 0.5237,   1994: 0.5467,
#   1995: 0.5494,   1996: 0.5574,   1997: 0.5327,   1998: 0.5229,   1999: 0.5152,
#   2000: 0.5063,   2001: 0.5456,   2002: 0.5242,   2003: 0.5239,   2004: 0.5821,
#   2005: 0.5401,   2006: 0.5552,   2007: 0.5609,   2008: 0.5454,   2009: 0.5443,
#   2010: 0.5250,   2011: 0.5305,   2012: 0.5465,   2013: 0.5812,   2014: 0.5750,   
#   2015: 0.5927
# }

In [15]:


'''--------------------------------- IMAGE PROCESSING ---------------------------------'''
def getMTR_area_constant_thresh(fips):
    the_county = counties.filterMetadata('FIPS', 'equals', fips);
   



    MTR_area = [[],[]];

    for year in range(1984,2016): # Years of interest for the study

        # Determine what imagery dataset to use, based off year loop; and what threshold to use
        if year <= 2011:
            imagery = ee.ImageCollection("LANDSAT/LT5_L1T_ANNUAL_GREENEST_TOA");
            NDVIbands = 43;

        elif year == 2012:
            imagery = ee.ImageCollection("LANDSAT/LE7_L1T_ANNUAL_GREENEST_TOA");
            NDVIbands = 43;

        elif year >= 2013:
            imagery = ee.ImageCollection("LANDSAT/LC8_L1T_ANNUAL_GREENEST_TOA");
            NDVIbands = 54; # Because different bands are needed for LS8

        # Select specific year for analysis
        yearImg = ee.Image(imagery.filterDate(str(year)+"-01-01", str(year)+"-12-31").first()).clip(the_county);

        # Calculate NDVI (using normalizedDifference function; select correct bands per sensor)
        if NDVIbands == 54:
            NDVI = yearImg.normalizedDifference(["B5","B4"]).clip(the_county);
        else:
            NDVI = yearImg.normalizedDifference(["B4","B3"]).clip(the_county);


        # Create a mask of areas that ARE NOT mines (value of 1 to locations where NDVI is <= threshold)
        lowNDVI = NDVI.where(NDVI.lte(NDVI_Threshold),1).where(NDVI.gt(NDVI_Threshold),0);

        # Create binary image containing the intersection between the LowNDVI and anywhere the inverted mask is 1
        MTR = lowNDVI.And(mask_input_60m_2015.eq(0));

        # Erode/dilate MTR sites to remove outliers (pixel clean-up)
        MTR_eroded_dialated_dialated_eroded = MTR.reduceNeighborhood(ee.Reducer.max(), ee.Kernel.euclidean(30, 'meters'))\
        .reduceNeighborhood(ee.Reducer.min(), ee.Kernel.euclidean(60, 'meters'))\
        .reduceNeighborhood(ee.Reducer.max(), ee.Kernel.euclidean(30, 'meters'));

        MTR_masked = MTR_eroded_dialated_dialated_eroded.updateMask(MTR_eroded_dialated_dialated_eroded);


        #  Get a pixel area image, which will apply to any scale you provide
        Area = ee.Image.pixelArea();

        # The area calculation, which currently burns out the server
        areaAll = MTR_masked.multiply(Area).reduceRegion(reducer=ee.Reducer.sum(),geometry=the_county, 
                                                          scale= 30,crs= 'EPSG:3857',maxPixels= 1e9)
        areaKmSq = ee.Number(areaAll.get(MTR_masked.bandNames().getInfo()[0])).divide(1000*1000);

        # Add these areas and their corresponding year to 2D array
        MTR_area[0].append(year);
        MTR_area[1].append(areaKmSq);

    return MTR_area

In [ ]:
for year in range(1972, 2015):
    
  // Determine what imagery dataset to use, based off year loop
    if (year <= 1974):
        imagery = ee.ImageCollection("LANDSAT/LM1_L1T");  // Landsat 1: 1972 - 1974
        NDVIbands = 75;
        outlierValue = 250;
  
    elif year > 1974 and year <= 1983:
        imagery = ee.ImageCollection("LANDSAT/LM2_L1T");  // Landsat 2: 1975 - 1982 [omit 1983]
        NDVIbands = 75;
        outlierValue = 250;
  
    elif year > 1983 and year <= 2011:
        imagery = ee.ImageCollection("LANDSAT/LT5_L1T");  // Landsat 5: 1984 - 2011
        NDVIbands = 43;
        outlierValue = 250;
  
    elif year == 2012:
        imagery = ee.ImageCollection("LANDSAT/LE7_L1T");  // Landsat 7: 2012
        NDVIbands = 43;
        outlierValue = 250;
  
    elif year >= 2013:
        imagery = ee.ImageCollection("LANDSAT/LC8_L1T");  // Landsat 8: 2013 - 
        NDVIbands = 54; // Because different bands are needed for LS8
        outlierValue = 64256;
        rgb_vis = {min:0, max:0.25, bands:['B4','B3','B2']};
  

     #    Calculate NDVI and create greenest pixel composite 
      #    (using normalizedDifference function; select correct bands per sensor)
    if (NDVIbands == 75){
        if year >= 1972 and year <= 1973: continue
        elif (year == 1974):
              yearImgs = imagery.filterDate("1972-01-01", "1974-12-31").filterBounds(studyArea).map(cleaner).map(earlyLScleaner);
              NDVI = yearImgs.select("nd").qualityMosaic("nd")
        
        elif (year >= 1975 and year <= 1976: continue
        elif (year == 1977):
             yearImgs = imagery.filterDate("1975-01-01", "1977-12-31").filterBounds(studyArea).map(cleaner).map(earlyLScleaner);
             NDVI = yearImgs.select("nd").qualityMosaic("nd")
        
        elif (year >= 1978 and year <= 1979: continue
              
        elif (year == 1980):
            yearImgs = imagery.filterDate("1978-01-01", "1980-12-31").filterBounds(studyArea).map(cleaner).map(earlyLScleaner);
            NDVI = yearImgs.select("nd").qualityMosaic("nd")
        elif (year == 1981){continue;}
        elif year == 1982:
            yearImgs = imagery.filterDate("1981-01-01", "1982-12-31").filterBounds(studyArea).map(cleaner).map(earlyLScleaner);
            NDVI = yearImgs.select("nd").qualityMosaic("nd");
    
        elif year == 1983: continue
  
  
    elsif NDVIbands == 54:
       # Select specific year for analysis and clean raw input data
        yearImgs = imagery.filterDate(str(year)+"-01-01", str(year)+"-12-31").filterBounds(studyArea).map(cleaner);
        ndCalc = yearImgs.map(function(image){
        calc = image.normalizedDifference(["B5","B4"]).clip(studyArea);
        return image.addBands(calc);
    });
    NDVI = ndCalc.select("nd").qualityMosaic("nd");
  }
  else if (NDVIbands == 43){
    // Select specific year for analysis and clean raw input data
    yearImgs = imagery
      .filterDate(year+"-01-01", year+"-12-31")
      .filterBounds(studyArea)
      .map(cleaner);
    ndCalc = yearImgs.map(function(image){
      calc = image.normalizedDifference(["B4","B3"]).clip(studyArea);
      return image.addBands(calc);
    });
    NDVI = ndCalc.select("nd").qualityMosaic("nd");
  }
  
  // Create a mask of areas that ARE NOT mines (value of 1 to locations where NDVI is <= threshold)
  lowNDVI = NDVI.where(NDVI.lte(NDVI_Threshold[year]),1).where(NDVI.gt(NDVI_Threshold[year]),0);
  
  // Create binary image containing the intersection between the LowNDVI and anywhere the inverted mask is 1
  MTR = lowNDVI.and(mask_input_60m_2015.eq(0));
  
  // Erode/dilate MTR sites to remove outliers (pixel clean-up)
  MTR_cleaning = MTR
    .reduceNeighborhood(ee.Reducer.min(), ee.Kernel.euclidean(30, 'meters'))  // erode
    .reduceNeighborhood(ee.Reducer.max(), ee.Kernel.euclidean(60, 'meters'))  // dilate
    //.reduceNeighborhood(ee.Reducer.max(), ee.Kernel.euclidean(60, 'meters'))  // dilate
    .reduceNeighborhood(ee.Reducer.min(), ee.Kernel.euclidean(30, 'meters')); // erode

  // Mask MTR by the erosion/dilation, and by mining permit locations buffered 1 km
  MTR_masked = MTR_cleaning.updateMask(MTR_cleaning).updateMask(miningPermits);

  /*--------------------------------- IMAGE VISUALIZATION ---------------------------------*/
  
  // Set color palette by year (http://colorbrewer2.org/?type=sequential&scheme=OrRd&n=8)
  // Currently every five years are a different color (dark -> light over time)
  if (year <= 1977){
    palette = "990000";
  }
  else if (year > 1977 and year <= 1982){
    palette = "d7301f";
  }
  else if (year > 1982 and year <= 1988){
    palette = "ef6548"
  }
  else if (year > 1988 and year <= 1993 ){
    palette = "fc8d59";
  }
  else if (year > 1993 and year <= 1998){
    palette = "fdbb84";
  }
  else if (year > 1998 and year <= 2003){
    palette = "fdd49e";
  }
  else if (year > 2003 and year <= 2008){
    palette = "fee8c8";
  }
  else if (year > 2008){
    palette = "fff7ec";
  }
  
  // Add each layer to the map; 
  // only years divisible by 5 turned on by default because this takes a while to display
  if (year % 5 === 0){
    Map.addLayer(MTR_masked, {palette: palette}, ("MTR "+ year), true);
  }
  else {
    Map.addLayer(MTR_masked, {palette: palette}, ("MTR "+ year), false);
  }
  
  /*--------------------------------- AREA CALCULATION ---------------------------------
  
  // Get a pixel area image, which will apply to any scale you provide
  Area = ee.Image.pixelArea();
  // The area calculation, which currently burns out the server
  areaAll = MTR_masked.multiply(Area).reduceRegion({
    reducer: ee.Reducer.sum(),
    geometry: geometryI,
    scale: 30,
    crs: 'EPSG:3857', // Kroodsma did not include this option
    maxPixels: 1e9
  });
  areaKmSq = ee.Number(areaAll.get('constant')).divide(1000*1000);
  
  // Add these areas and their corresponding year to 2D array
  MTR_area[0].push(year);
  MTR_area[1].push(areaKmSq);
  */
  
  /* -------------------------------- EXPORTING ------------------------------------------- */
  
  if (exportImages === true) {
  // Set CRS and transform; create rectangular boundaries for exporting
  //crs = MTR_masked.projection().atScale(30).getInfo().crs;
  //transform = MTR_masked.projection().atScale(30).getInfo().transform;

  /*// Export entire study region to GDrive (many images; one per year!)
  Export.image.toDrive({
      image: MTR_masked.unmask(0),
      description: "MTR_"+year,
      region: exportbounds,
      scale: 900,
      //crs: crs,
      //crsTransform: transform
    });*/
    
  // OR, export one or some of the subregions to GDrive (many images!) 
  Export.image.toDrive({
      image: MTR_masked.unmask(0),
      description: "MTR"+year+"reg1",
      region: geometryI,
      maxPixels: 1e9,
      scale: 30,
      //crs: crs,
      //crsTransform: transform
    });
  Export.image.toDrive({
      image: MTR_masked.unmask(0),
      description: "MTR"+year+"reg2",
      region: geometryII,
      maxPixels: 1e9,
      scale: 30,
      //crs: crs,
      //crsTransform: transform
    });
  Export.image.toDrive({
      image: MTR_masked.unmask(0),
      description: "MTR"+year+"reg3",
      region: geometryIII,
      maxPixels: 1e9,
      scale: 30,
      //crs: crs,
      //crsTransform: transform
    });
      
  Export.image.toDrive({
      image: MTR_masked.unmask(0),
      description: "MTR"+year+"reg4",
      region: geometryIV,
      maxPixels: 1e9,
      scale: 30,
      //crs: crs,
      //crsTransform: transform
    });
  }  
    
  // Add each layer to a list, so as to build an ImageCollection for Video
  allMTR_list.push(MTR_masked);
}

In [7]:

'''--------------------------------- IMAGE PROCESSING ---------------------------------'''
def getMTR_area_60(fips):
    the_county = counties.filterMetadata('FIPS', 'equals', fips);
    
    MTR_area = [[],[]];

    for year in range(1984,2016): # Years of interest for the study

        # Determine what imagery dataset to use, based off year loop; and what threshold to use
        if year <= 2011:
            imagery = ee.ImageCollection("LANDSAT/LT5_L1T_ANNUAL_GREENEST_TOA");
            NDVIbands = 43;

        elif year == 2012:
            imagery = ee.ImageCollection("LANDSAT/LE7_L1T_ANNUAL_GREENEST_TOA");
            NDVIbands = 43;

        elif year >= 2013:
            imagery = ee.ImageCollection("LANDSAT/LC8_L1T_ANNUAL_GREENEST_TOA");
            NDVIbands = 54; # Because different bands are needed for LS8

        # Select specific year for analysis
        yearImg = ee.Image(imagery.filterDate(str(year)+"-01-01", str(year)+"-12-31").first()).clip(the_county);

        # Calculate NDVI (using normalizedDifference function; select correct bands per sensor)
        if NDVIbands == 54:
            NDVI = yearImg.normalizedDifference(["B5","B4"]).clip(the_county);
        else:
            NDVI = yearImg.normalizedDifference(["B4","B3"]).clip(the_county);


        # Create a mask of areas that ARE NOT mines (value of 1 to locations where NDVI is <= threshold)
        lowNDVI = NDVI.where(NDVI.lte(NDVI_Threshold),1).where(NDVI.gt(NDVI_Threshold),0);

        # Create binary image containing the intersection between the LowNDVI and anywhere the inverted mask is 1
        MTR = lowNDVI.And(mask_input_60m_2015.eq(0));

        # Erode/dilate MTR sites to remove outliers (pixel clean-up)
        MTR_eroded_dialated_dialated_eroded = MTR.reduceNeighborhood(ee.Reducer.max(), ee.Kernel.euclidean(60, 'meters'))\
        .reduceNeighborhood(ee.Reducer.min(), ee.Kernel.euclidean(120, 'meters'))\
        .reduceNeighborhood(ee.Reducer.max(), ee.Kernel.euclidean(60, 'meters'));

        MTR_masked = MTR_eroded_dialated_dialated_eroded.updateMask(MTR_eroded_dialated_dialated_eroded);


        #  Get a pixel area image, which will apply to any scale you provide
        Area = ee.Image.pixelArea();

        # The area calculation, which currently burns out the server
        areaAll = MTR_masked.multiply(Area).reduceRegion(reducer=ee.Reducer.sum(),geometry=the_county, 
                                                          scale= 30,crs= 'EPSG:3857',maxPixels= 1e9)
        areaKmSq = ee.Number(areaAll.get(MTR_masked.bandNames().getInfo()[0])).divide(1000*1000);


        # Add these areas and their corresponding year to 2D array
        MTR_area[0].append(year);
        MTR_area[1].append(areaKmSq);

    return MTR_area

In [8]:
def get_area_list(MTR_area):
    county_areas = []
    for i in range(len(MTR_area[1])):
        try:
        print MTR_area[0][i],"\t",
        try:
            the_area = MTR_area[1][i].getInfo()
            county_areas.append(the_area)
        except:
            time.sleep(20)
            the_area = MTR_area[1][i].getInfo()
            county_areas.append(the_area)
        print the_area
        time.sleep(15)
    return county_areas

In [9]:
county_areas = {}


In [14]:
for county in wv_counties:
    if county not in county_areas:
        print county
        fips = wv_counties[county]
        MTR_area = getMTR_area_60(fips)
        area_list = get_area_list(MTR_area)
        county_areas[county] = area_list

Kanawha
1984 	7.27869814496
1985 	8.28391128967
1986 	7.99854974335
1987 	10.1137607228
1988 	11.626124019
1989 	108.554576903
1990 	12.169379804
1991 	14.4988228085
1992 	12.5996292732
1993 	11.7116152402
1994 	13.1837384681
1995 	10.4762710424
1996 	8.15755616986
1997 	10.717355843
1998 	10.1463300873
1999 	11.6422616722
2000 	10.9987261334
2001 	11.4811927718
2002 	13.9812724379
2003 	11.9809390399
2004 	12.2797889207
2005 	14.3454362805
2006 	14.8869988939
2007 	17.0549876244
2008 	16.0252961165
2009 	17.2607920317
2010 	14.3284259799
2011 	14.5902525085
2012 	16.0625308836
2013 	12.8424401666
2014 	12.7485604633
2015 	13.8261528462
Logan
1984 	6.78154855072
1985 	5.07361943414
1986 	6.05860308301
1987 	7.81334083923
1988 	10.524796748
1989 	

EEException: User memory limit exceeded.

In [15]:

'''--------------------------------- IMAGE PROCESSING ---------------------------------'''
def getMTR_area_120(fips):
    the_county = counties.filterMetadata('FIPS', 'equals', fips);
    
    MTR_area = [[],[]];

    for year in range(1984,2016): # Years of interest for the study

        # Determine what imagery dataset to use, based off year loop; and what threshold to use
        if year <= 2011:
            imagery = ee.ImageCollection("LANDSAT/LT5_L1T_ANNUAL_GREENEST_TOA");
            NDVIbands = 43;

        elif year == 2012:
            imagery = ee.ImageCollection("LANDSAT/LE7_L1T_ANNUAL_GREENEST_TOA");
            NDVIbands = 43;

        elif year >= 2013:
            imagery = ee.ImageCollection("LANDSAT/LC8_L1T_ANNUAL_GREENEST_TOA");
            NDVIbands = 54; # Because different bands are needed for LS8

        # Select specific year for analysis
        yearImg = ee.Image(imagery.filterDate(str(year)+"-01-01", str(year)+"-12-31").first()).clip(the_county);

        # Calculate NDVI (using normalizedDifference function; select correct bands per sensor)
        if NDVIbands == 54:
            NDVI = yearImg.normalizedDifference(["B5","B4"]).clip(the_county);
        else:
            NDVI = yearImg.normalizedDifference(["B4","B3"]).clip(the_county);


        # Create a mask of areas that ARE NOT mines (value of 1 to locations where NDVI is <= threshold)
        lowNDVI = NDVI.where(NDVI.lte(NDVI_Threshold),1).where(NDVI.gt(NDVI_Threshold),0);

        # Create binary image containing the intersection between the LowNDVI and anywhere the inverted mask is 1
        MTR = lowNDVI.And(mask_input_60m_2015.eq(0));

        # Erode/dilate MTR sites to remove outliers (pixel clean-up)
        MTR_eroded_dialated_dialated_eroded = MTR.reduceNeighborhood(ee.Reducer.max(), ee.Kernel.euclidean(60*2, 'meters'))\
        .reduceNeighborhood(ee.Reducer.min(), ee.Kernel.euclidean(120*2, 'meters'))\
        .reduceNeighborhood(ee.Reducer.max(), ee.Kernel.euclidean(60*2, 'meters'));

        MTR_masked = MTR_eroded_dialated_dialated_eroded.updateMask(MTR_eroded_dialated_dialated_eroded);


        #  Get a pixel area image, which will apply to any scale you provide
        Area = ee.Image.pixelArea();

        # The area calculation, which currently burns out the server
        areaAll = MTR_masked.multiply(Area).reduceRegion(reducer=ee.Reducer.sum(),geometry=the_county, 
                                                          scale= 30,crs= 'EPSG:3857',maxPixels= 1e9)
        areaKmSq = ee.Number(areaAll.get(MTR_masked.bandNames().getInfo()[0])).divide(1000*1000);


        # Add these areas and their corresponding year to 2D array
        MTR_area[0].append(year);
        MTR_area[1].append(areaKmSq);

    return MTR_area

In [ ]:
county_areas2 = {}
for county in wv_counties:
    if county not in county_areas2:
        print county
        fips = wv_counties[county]
        MTR_area = getMTR_area_120(fips)
        try:
            area_list = get_area_list(MTR_area)
            county_areas2[county] = area_list
        except:
            print "fail for", county

Kanawha
1984 	9.11170358234
1985 	9.21859120807
1986 	9.01843446509
1987 	12.7117477738
1988 	14.3399619163
1989 	128.085904624
1990 	14.5465105819
1991 	17.5936161973
1992 	15.6143896973
1993 	14.8250257872
1994 	16.1505786734
1995 	12.3519103941
1996 	9.45473438672
1997 	12.0775450353
1998 	11.0436583371
1999 	12.3097172758
2000 	11.5121852794
2001 	12.8123648444
2002 	16.0281477527
2003 	fail for Kanawha
Logan
1984 	8.68781332544
1985 	6.13104911206
1986 	7.58968417401
1987 	9.77782244598
1988 	12.8893387217
1989 	80.6652979287
1990 	18.4482541434
1991 	30.1649269652
1992 	22.5995534563
1993 	24.8593627413
1994 	52.1347533427
1995 	34.7523619612
1996 	29.2465947054
1997 	30.9258125305
1998 	33.3884986431
1999 	43.3978268651
2000 	37.4948944093
2001 	31.779981661
2002 	37.2655396324
2003 	26.8775398318
2004 	24.8599043286
2005 	36.0920420459
2006 	33.7407957938
2007 	45.5124865206
2008 	41.0970933096
2009 	49.2468149294
2010 	39.4641160046
2011 	34.9045858763
2012 	33.1408162195
2013